In [53]:
def intersect_trop_lines(first,second):
    a = first[0]
    b = first[1]
    c = second[0]
    d = second[1]
    #anchored at (c,d)
    if a>c and b>d and a>b: #1
        return [c+b,d+b]
    if a>c and b>d and a<b: #2
        return [c+a,d+a]
    if a>c and b>d and a==b:
        return [c+b-d,b]
    if a<c and b>d: #3
        return [a,d]
    if a<c and b<d and a<b: #4
        return [a+d-b,d]
    if a<c and b<d and a>b: #5
        return [c,b+c-a]
    if a<c and b<d and a==b:
        return [c,b+c-a]
    if a>c and d>b: #6
        return [c,b]
    #anchored at (a,b)
    if c>a and d>b and c>d: #1
        return [a+d,b+d]
    if c>a and d>b and c<d: #2
        return [a+c,b+c]
    if c>a and d>b and c==d:
        return [a+d-b,d]
    if c<a and d>b: #3
        return [c,b]
    if c<a and d<b and c<d: #4
        return [c+b-d,b]
    if c<a and d<b and c>d: #5
        return [a,d+a-c]
    if c<a and d<b and c==d:
        return [a,d+a-c]
    if c>a and b>d: #6
        return [a,d]
    
    
def compute_pseudovertices(vertices):
    pseudovertices = copy(vertices)
    edges = []
    for pair in Subsets(range(len(vertices)),2):
        if intersect_trop_lines(vertices[pair[0]],vertices[pair[1]]):
            pseudovertices.append(intersect_trop_lines(vertices[pair[0]],vertices[pair[1]]))
            index = len(pseudovertices)-1
            edges.append([pair[0],index])
            edges.append([pair[1],index])
        else:
            edges.append(list(pair))
    return {'pseudo': pseudovertices, 'edges': edges, 'vertices': vertices}


def tangent_cone(polytron, vertex_index):
    
    def primitive_vector_in_edge_direction(direction):
        x = direction[0]
        y = direction[1]
        if x==0 and y>0:
            primitive = [ 0, 1]
        elif x==0 and y<0:
            primitive = [ 0,-1]
        elif x>0 and y>0:
            primitive = [ 1, 1]
        elif x>0 and y==0:
            primitive = [ 1, 0]
        elif x<0 and y==0:
            primitive = [-1, 0]
        elif x<0 and y<0:
            primitive = [-1,-1]
        else:
            print 'not assigned'
        return primitive
    
    ##compute the generating function of the tangent cone at the given vertex
    vertex = polytron['pseudo'][vertex_index]
    
    
    #find neighbours of vertex
    incident_edges_index = [edge for edge in polytron["edges"] if vertex_index in edge]
    neighbours_index = []
    for edge_index in incident_edges_index:
        for index in edge_index:
            if index==vertex_index:
                continue
            else:
                neighbours_index.append(index)    
    neighbours = [ polytron['pseudo'][index] for index in neighbours_index ]
    
    #compute edge-directions
    edge_directions = [ [neighbour[0]-vertex[0],neighbour[1]-vertex[1]] for neighbour in neighbours ]
    
    primitives=[]
    #find primitive vector in edge-direction
    for direction in edge_directions:
        primitives.append(primitive_vector_in_edge_direction(direction))
    
    
    t1, t2, n = var('t1, t2, n')
    return ( t1^(n*vertex[0])* t2^(n*vertex[1]) )  / ( (1 - t1^primitives[0][0] * t2^primitives[0][1]) * (1 - t1^primitives[1][0] * t2^primitives[1][1])  )

def gen_fct(polytron):
    #for vertex_index in range(len(polytron["pseudo"])):
        #print tangent_cone(polytron,vertex_index)  
    return sum([tangent_cone(polytron,vertex_index) for vertex_index in range(len(polytron["pseudo"]))])

def ehrhart(polytron,evaluation):
    solutions = [gen_fct(polytron)(a=evaluation[0],b=evaluation[1]).factor()(n=k).factor()(t1=1,t2=1) for k in range(1,4)]
    mat = matrix([ [1,1,1],
                   [4,2,1],
                   [9,3,1] ])
    coeffs = mat.solve_right(vector(solutions))
    t = var('t')
    return sum(coeffs[i]*t^(2-i) for i in range(3))
        


In [54]:
a, b = var('a, b')
forget(assumptions())
assume( a>-1, -a-3<b, -b - 2<0)

vertices = [[b,1],[-a-3,-a],[b,-a]]
polytron = compute_pseudovertices(vertices)
#print polytron

#print gen_fct(polytron).numerator_denominator() 
generating_f = gen_fct(polytron).factor()
print generating_f
ehrhart(polytron,[0,0])

(t1^(a*n + b*n + 3*n + 2)*t2^(a*n + n + 2) - t1^(a*n + b*n + 3*n + 2)*t2 - t1^(a*n + n + 1)*t2^(a*n + n + 2) - t1^(a*n + b*n + 3*n + 1)*t2^(a*n + n + 1) + t1^(a*n + n + 1)*t2^(a*n + n + 1) + t1^(a*n + b*n + 3*n + 1) + t2 - 1)*t1^(-a*n - 3*n)/((t1*t2 - 1)*(t1 - 1)*(t2 - 1)*t2^(a*n))


5/2*t^2 + 7/2*t + 1

In [45]:
#substitute all variables by (1-t)
t = var('t')
for j in range(1,3):
     generating_f = generating_f.subs({var('t'+str(j)):1-t})
    #generating_f = generating_f.subs({var('t'+str(j)):1-j*t})
print generating_f

((t - 1)*(-t + 1)^(a*n + b*n + 3*n + 2) + (-t + 1)^(a*n + b*n + 3*n + 2)*(-t + 1)^(a*n + n + 2) - (-t + 1)^(a*n + b*n + 3*n + 1)*(-t + 1)^(a*n + n + 1) - (-t + 1)^(a*n + n + 2)*(-t + 1)^(a*n + n + 1) - t + (-t + 1)^(2*a*n + 2*n + 2) + (-t + 1)^(a*n + b*n + 3*n + 1))*(-t + 1)^(-a*n - 3*n)/(((t - 1)^2 - 1)*t^2*(-t + 1)^(a*n))


In [50]:
#l'hopital
p = generating_f.numerator()
q = generating_f.denominator()
alpha = 0
p_deriv= p
q_deriv = q
while true:
    p_deriv = p_deriv.derivative(t)
    q_deriv = q_deriv.derivative(t)
    alpha += 1
    if not (p_deriv(t=0)==0 or q_deriv(t)==0):
        break
result = p_deriv(t=0)/q_deriv(t=0)
print result.simplify_full()

1/2*(a^2 + 2*(a + 1)*b + 6*a + 5)*n^2 + 1/2*(3*a + 2*b + 7)*n + 1


In [51]:
result(a=1,b=0).simplify_full()

6*n^2 + 5*n + 1